In [1]:
import tushare as ts
import pandas as pd
from datetime import date, datetime,timedelta
import time
from tqdm.notebook import tqdm_notebook

#设置完整显示数据
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 
              500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

#tushare.org注册获得的token
pro = ts.pro_api('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
ts.set_token('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')


In [3]:
#设置时间范围
start_date = '20200102'
end_date = '20220216'
today = '20220315'

#输入股票代码、同行竞争对手
stock_selections = ['601012']
competitors = ['金辰股份', '隆基股份', '中环股份', '迈为股份', '高测股份', '上机数控', '罗博特科', '京运通', '*ST天龙', '精功科技', '先导智能', '晶盛机电', '北方华创', '捷佳伟创', '天宸股份', '聆达股份', '*ST大洲', '江泉实业', '*ST大港', '华金资本', '中国宝安', '悦达投资', '浙江富润', '数源科技', '奥园美谷', '东方日升', '爱旭股份', '通威股份', '横店东磁', '晶澳科技', '协鑫集成', '中来股份', '亿晶光电', '天合光能', '向日葵', '正泰电器', '航天机电', '拓日新能', '大全能源', '安彩高科', '福莱特', '振江股份', '中信博', '新纶科技', '赛伍技术', '乐凯胶片', '海优新材', '爱康科技', '清源股份', '回天新材', '道明光学', '南玻A', '亚玛顿', '易成新能', '福斯特', '明冠新材']

## （1）同行业类型股票参考

In [13]:
stock_basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name')

if competitors[0].isnumeric():
    stock_selections = stock_basic[stock_basic['symbol'].isin(competitors)]
else:
    stock_selections = stock_basic[stock_basic['name'].isin(competitors)]
    
stock_selections.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 7 to 4621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ts_code  53 non-null     object
 1   symbol   53 non-null     object
 2   name     53 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [9]:
def days(isse_date,trade_date):
    is_date = datetime.strptime(isse_date, "%Y%m%d")
    tr_date = datetime.strptime(trade_date, "%Y%m%d")
    delta =  tr_date - is_date
    return delta.days
    
def find_max_min_stock_data(stock_data, latest_date):
    #获得最新日期的数据
    latest_date_row = stock_data[stock_data['trade_date'] == latest_date].iloc[0]
        
    #最大值
    maxValueIndex = stock_data['close'].idxmax()
    max_row = stock_data.loc[maxValueIndex]

    #find minimum data after maximum point
    #最小值
    min_stock_data = stock_data[stock_data['trade_date'] > max_row['trade_date']]
    
    minValueIndex = min_stock_data['close'].idxmin()
    min_row = min_stock_data.loc[minValueIndex]
        
    return latest_date_row, max_row, min_row

        
def qfq_daily_data(ts_code, start_date, end_date, retry_count=3, pause=15):
    for _ in range(retry_count):
        try:
            df = ts.pro_bar(ts_code=ts_code, adj='qfq', freq ='D', start_date=start_date, end_date=end_date)
        except:
            time.sleep(pause)
        else:
            return df

In [14]:
data = []
for index, row in tqdm_notebook(stock_selections.iterrows(), total=stock_selections.shape[0]):
    try:
        #获取股票日数据
        _daily_data = qfq_daily_data(row['ts_code'], start_date, end_date)
        _daily_data = _daily_data.drop_duplicates()
         
        #计算最新、最大、最小值
        latest_date_row, max_row, min_row = find_max_min_stock_data(_daily_data, end_date)
        
        #最大/小值_交易日
        max_date = max_row['trade_date']
        min_date = min_row['trade_date']
        
        #计算最大、最小值对应的PE
        pe_max = pro.daily_basic(ts_code=row['ts_code'], trade_date = max_date, fields='pe, pe_ttm').iloc[0]
        pe_min = pro.daily_basic(ts_code=row['ts_code'],trade_date = min_date, fields='pe, pe_ttm').iloc[0]
                
        
        
        #保存数据至新list
        ts_data = {}
        ts_data['ts_code'] = row['ts_code']
        ts_data['名称'] = row['name']
        ts_data['数据起始日期'] = start_date
        ts_data['数据截至日期'] = end_date
        
        ts_data['最新_交易日'] = latest_date_row['trade_date']
        ts_data['最新_收盘价'] = latest_date_row['close']
        ts_data['最大值_交易日'] = max_date
        ts_data['最大值_收盘价'] = max_row['close']
        ts_data['最大值_PE'] = pe_max['pe']
        ts_data['最大值_PE_TTM'] = pe_max['pe_ttm']
        
        ts_data['最小值_交易日'] = min_row['trade_date']
        ts_data['最小值_收盘价'] = min_row['close']
        ts_data['最小值_PE'] = pe_min['pe']
        ts_data['最小值_PE_TTM'] = pe_min['pe_ttm']
        
        ts_data['最大回撤_横跨周期'] = days(max_row['trade_date'], min_row['trade_date'])
        #最到位之后的回撤
        ts_data['最大回撤'] = (min_row['close']/max_row['close'] - 1) * 100
        
        #最到位之后的到最新收盘价的涨跌幅
        ts_data['最高_最新收盘价_涨跌幅'] = (latest_date_row['close']/max_row['close'] - 1) * 100
        data.append(ts_data)
        
    except Exception as e:
        print(e)
        pass


data_df = pd.DataFrame(data) 


  0%|          | 0/53 [00:00<?, ?it/s]

In [15]:
data_df['最大回撤_日化'] = data_df['最大回撤'].div(data_df['最大回撤_横跨周期'],'')

mv_uptodate = pro.daily_basic(trade_date = today, fields='ts_code,total_mv, pe, pe_ttm')
mv_uptodate['total_mv'] = mv_uptodate['total_mv']/10000 
mv_uptodate = mv_uptodate.rename(columns={col: col+'_{}'.format(today) 
                        for col in mv_uptodate.columns if col not in ['ts_code']})

data_df=data_df.merge(mv_uptodate, on=['ts_code'])

In [16]:
data_df.sort_values(by='最大回撤', ascending=True)

,ts_code,名称,数据起始日期,数据截至日期,最新_交易日,最新_收盘价,最大值_交易日,最大值_收盘价,最大值_PE,最大值_PE_TTM,最小值_交易日,最小值_收盘价,最小值_PE,最小值_PE_TTM,最大回撤_横跨周期,最大回撤,最高_最新收盘价_涨跌幅,最大回撤_日化,pe_20220315,pe_ttm_20220315,total_mv_20220315
3,000615.SZ,奥园美谷,20200102,20220216,20220216,8.32,20210601,29.09,NaN,NaN,20220127,8.11,NaN,36.75,240,-72.12,-71.40,-0.30,NaN,32.40,55.85
41,603185.SH,上机数控,20200102,20220216,20220216,127.40,20210902,349.66,181.14,78.23,20220211,119.60,61.95,20.65,162,-65.80,-63.56,-0.41,82.08,26.65,436.12
48,688408.SH,中信博,20200102,20220216,20220216,103.00,20210730,268.00,127.40,128.89,20220211,92.39,43.92,76.12,196,-65.53,-61.57,-0.33,39.57,358.28,112.96
43,603396.SH,金辰股份,20200102,20220216,20220216,74.61,20210827,201.30,282.84,249.00,20220211,70.33,99.07,84.71,168,-65.06,-62.94,-0.39,100.02,85.53,82.58
24,300724.SZ,捷佳伟创,20200102,20220216,20220216,76.14,20210831,205.99,136.94,97.89,20220214,74.17,49.39,38.44,167,-63.99,-63.04,-0.38,49.52,38.54,258.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,002056.SZ,横店东磁,20200102,20220216,20220216,16.43,20211214,21.89,35.13,28.74,20220125,15.56,24.97,20.43,42,-28.92,-24.94,-0.69,19.41,19.41,217.49
26,300757.SZ,罗博特科,20200102,20220216,20220216,51.00,20211213,69.61,NaN,NaN,20220214,50.08,NaN,NaN,63,-28.06,-26.73,-0.45,NaN,NaN,46.75
10,002459.SZ,晶澳科技,20200102,20220216,20220216,81.51,20211122,101.70,107.92,106.44,20220210,74.63,79.21,78.13,80,-26.62,-19.85,-0.33,92.54,91.27,"1,394.12"
29,600151.SH,航天机电,20200102,20220216,20220216,11.18,20211231,13.85,115.83,79.45,20220208,10.90,91.16,62.52,39,-21.30,-19.28,-0.55,82.46,56.56,141.42


In [ ]:
data_df.to_excel('同行业跌幅排行榜.xlsx')